<a href="https://colab.research.google.com/github/sayma-yousuf/IssueDetection/blob/main/Update_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
import os

In [ ]:
#!pip install transformers
#!pip install torch
#!pip install absl-py
#!pip install ml-dtypes
#!pip install gast
#!pip install astunparse
#!pip install termcolor
#!pip install opt_einsum   flatbuffers
#!pip install flatbuffers  

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, BertTokenizerFast, BertForSequenceClassification, RobertaTokenizerFast, RobertaForSequenceClassification, AdamW
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample

# Load your dataset
#df = pd.read_csv('/root/workspace/aka_project/Naikdil/annotatted_dataset.csv')
df = pd.read_csv('ChatGPT_Human_annotated_dataset.csv')
# Compute class distribution
class_counts = df['category'].value_counts()
print(class_counts)

In [ ]:
# Convert categorical class labels to numeric codes
df['category_code'] = df['category'].astype('category').cat.codes

# Create a histogram of class distribution
plt.figure(figsize=(10, 6))
sns.histplot(df, x='category_code', discrete=True, palette='Set2')
plt.title('Class Distribution')
plt.xlabel('Category')
plt.ylabel('Frequency')
plt.xticks(ticks=range(len(df['category'].astype('category').cat.categories)),
           labels=df['category'].astype('category').cat.categories,
           rotation=45)
plt.show()


In [ ]:
# Plot the class distribution as a pie chart
plt.figure(figsize=(8, 8))
plt.pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', startangle=140, colors=plt.get_cmap('tab20').colors)
plt.title('Review Distribution', fontsize=18, fontweight='bold')
plt.show()


In [ ]:
# Balance the dataset
df_majority = df[df['category'] == df['category'].value_counts().idxmax()]
df_minority = [df[df['category'] == cls] for cls in df['category'].unique() if cls != df['category'].value_counts().idxmax()]

# Upsample minority classes
df_minority_upsampled = [resample(minority, replace=True, n_samples=len(df_majority), random_state=42) for minority in df_minority]

# Combine majority class with upsampled minority classes
df_balanced = pd.concat([df_majority] + df_minority_upsampled)

# Shuffle the dataset
df = df_balanced.sample(frac=1, random_state=42)

# Display the class distribution
class_counts = df['category'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Review Distribution After Sampling', fontsize=18, fontweight='bold')
plt.show()

In [ ]:
# Convert categorical labels to numeric
df['category'] = df['category'].astype('category').cat.codes

if df['category'].dtype.name == 'category':
    df['category'] = df['category'].cat.codes

# Mapping from numeric to original labels
category_mapping = {
    0: 'Functionality issue',
    1: 'User Experience',
    2: 'Performance issue',
    3: 'Stability issue',
    4: 'Bug',
    5: 'Device Issue',
    6: 'Privacy Issue',
    7: 'User Interface',
    8: 'Customer Support Issues',
    9: 'Compatibility Issue',
    10: 'Network Issues',
    11: 'Installation Issue',
    12: 'Security issue',
}

# Apply the mapping
df['category'] = df['category'].map(category_mapping)

# Tokenizing using a simple approach (e.g., splitting the text into words)
df['token_length'] = df['Base_Reviews'].apply(lambda x: len(x.split()))

# Plot the distribution of token lengths with actual class names
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='token_length', hue='category', multiple='stack', palette='Set1', bins=50)
plt.title('Distribution of Review Length', fontsize=18, fontweight='bold')
plt.xlabel('Token Length')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Convert categorical class labels to numeric codes
df['category_code'] = df['category'].astype('category').cat.codes
df['category'] = df['category_code']

# Ensure the required package is installed
#%pip install transformers

# Initialize DistilBERT tokenizer
from transformers import DistilBertTokenizerFast
# Ensure the tokenizer is downloaded from the Hugging Face model hub
distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained('/root/workspace/aka_project/Naikdil/distilbert', local_files_only=True)

# Tokenize data for DistilBERT
def tokenize_data_distilbert(data):
    return distilbert_tokenizer(data['Base_Reviews'].tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='pt', return_attention_mask=True)
# Apply the tokenization function
tokenized_data_distilbert = tokenize_data_distilbert(df)

In [ ]:
class TextDatasetDistilBERT(Dataset):
    def __init__(self, inputs, labels, attention_masks):
        self.inputs = inputs
        self.labels = labels
        self.attention_masks = attention_masks

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx].clone().detach().to(torch.long),
            'attention_mask': self.attention_masks[idx].clone().detach().to(torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure labels are long
        }



In [ ]:
from transformers import get_linear_schedule_with_warmup

model_distilbert = DistilBertForSequenceClassification.from_pretrained('/root/workspace/aka_project/Naikdil/distilbert', num_labels=len(df['category'].unique()))
optimizer_distilbert = AdamW(model_distilbert.parameters(), lr=2e-5)
scheduler = None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_distilbert.to(device)
# Train model for DistilBert
def train_model_distilbert(train_loader, val_loader, epochs):
    # Initialize the learning rate scheduler
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer_distilbert, num_warmup_steps=0, num_training_steps=total_steps)

    history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': []}
    for epoch in range(epochs):
        model_distilbert.train()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        for batch in train_loader:
            optimizer_distilbert.zero_grad()
            inputs = batch['input_ids'].to(device)
            attention_masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model_distilbert(inputs, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer_distilbert.step()
            scheduler.step()  # Update learning rate

            total_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += labels.size(0)

        avg_train_loss = total_loss / len(train_loader)
        avg_train_accuracy = correct_predictions / total_samples
        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(avg_train_accuracy)

        # Validation
        model_distilbert.eval()
        val_loss = 0
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['input_ids'].to(device)
                attention_masks = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model_distilbert(inputs, attention_mask=attention_masks, labels=labels)
                val_loss += outputs.loss.item()
                preds = torch.argmax(outputs.logits, dim=1)
                correct_predictions += torch.sum(preds == labels).item()
                total_samples += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_accuracy = correct_predictions / total_samples
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(avg_val_accuracy)

        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_accuracy:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}')

    return model_distilbert, history

In [ ]:
kf = KFold(n_splits=5)
accuracies_distilbert = []
train_losses_distilbert = []
val_losses_distilbert = []
train_accuracies_distilbert = []
val_accuracies_distilbert = []

for train_idx, val_idx in kf.split(tokenized_data_distilbert['input_ids']):
    # Split tokenized data
    train_inputs_distilbert = tokenized_data_distilbert['input_ids'][train_idx]
    val_inputs_distilbert = tokenized_data_distilbert['input_ids'][val_idx]
    train_attention_masks_distilbert = tokenized_data_distilbert['attention_mask'][train_idx]
    val_attention_masks_distilbert = tokenized_data_distilbert['attention_mask'][val_idx]

    # Labels
    train_labels_distilbert = df['category'].iloc[train_idx].values
    val_labels_distilbert = df['category'].iloc[val_idx].values

    # Create datasets
    train_dataset_distilbert = TextDatasetDistilBERT(train_inputs_distilbert, train_labels_distilbert, train_attention_masks_distilbert)
    val_dataset_distilbert = TextDatasetDistilBERT(val_inputs_distilbert, val_labels_distilbert, val_attention_masks_distilbert)

    # Create dataloaders
    train_loader_distilbert = DataLoader(train_dataset_distilbert, batch_size=16, shuffle=True)
    val_loader_distilbert = DataLoader(val_dataset_distilbert, batch_size=16)

    # Train model
    model_distilbert, history_distilbert = train_model_distilbert(train_loader_distilbert, val_loader_distilbert, epochs=6)

    # Collect history
    train_losses_distilbert.extend(history_distilbert['train_loss'])
    val_losses_distilbert.extend(history_distilbert['val_loss'])
    train_accuracies_distilbert.extend(history_distilbert['train_accuracy'])
    val_accuracies_distilbert.extend(history_distilbert['val_accuracy'])



In [ ]:
# Accuracy calculation
model_distilbert.eval()
preds_distilbert, pred_probs_distilbert, true_labels_distilbert = [], [], []

with torch.no_grad():
    for batch in val_loader_distilbert:
        inputs = batch['input_ids'].to(device)
        attention_masks = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model_distilbert(inputs, attention_mask=attention_masks)

        # Get class probabilities
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_probs_distilbert.extend(probs.cpu().numpy())

        # Get predicted classes
        predictions = torch.argmax(probs, dim=1)
        preds_distilbert.extend(predictions.cpu().numpy())
        true_labels_distilbert.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy_distilbert = sum([pred == label for pred, label in zip(preds_distilbert, true_labels_distilbert)]) / len(true_labels_distilbert)
accuracies_distilbert.append(accuracy_distilbert)

print(f"DistilBERT Validation Accuracy: {accuracy_distilbert:.4f}")


In [ ]:
import os

# Save the model and tokenizer locally
distilbert_save_directory = './distilbert_model'  # current working directory
if not os.path.exists(distilbert_save_directory):
    os.makedirs(distilbert_save_directory)

model_distilbert.save_pretrained(distilbert_save_directory)
distilbert_tokenizer.save_pretrained(distilbert_save_directory)

print(f"DistilBert Model and Tokenizer saved to {distilbert_save_directory}")


In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch

# Load the model and tokenizer from local directory
distilbert_save_directory = './distilbert_model'
model_distilbert = DistilBertForSequenceClassification.from_pretrained(distilbert_save_directory)
distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained(distilbert_save_directory)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_distilbert.to(device)

# Define the class labels
# class_labels = ['Functionality and Features', 'Performance and Stability',
#                  'User Interface and UX', 'Compatibility and Device Issues',
#                  'Bug', 'Customer Support and Responsiveness', 'Security and Privacy Concerns','Network Issues','Installation Problem']
class_labels = ['Functionality issue', 'User Experience', 'Performance issue', 'Stability issue', 
                'Bug', 'Device Issue', 'Privacy Issue', 'User Interface', 'Customer Support Issues',
                'Compatibility Issue', 'Network Issues', 'Installation Issue', 'Security issue']
def classify_review_distilbert(review_text, tokenizer, model, device, class_labels):
    inputs = tokenizer(review_text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class_idx = torch.argmax(logits, dim=1).item()

    predicted_label = class_labels[predicted_class_idx]

    return predicted_label

# Example review
review_text = "I had a terrible time getting it installed on my tablet. I had to download it from three different sites. Then I had to install another program on my laptop only to be told my printer wasn't supported."

# Classify the review
predicted_label_distilbert = classify_review_distilbert(review_text, distilbert_tokenizer, model_distilbert, device, class_labels)
print(f"Predicted Label for DistilBert: {predicted_label_distilbert}")


In [ ]:
# Plot training and validation loss/accuracy
plt.figure(figsize=(14, 6))

# Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(train_losses_distilbert, label='Training Loss (DistilBert)')
plt.plot(val_losses_distilbert, label='Validation Loss (DistilBert)')
plt.title('Training and Validation Loss (DistilBert)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies_distilbert, label='Training Accuracy (DistilBert)')
plt.plot(val_accuracies_distilbert, label='Validation Accuracy (DistilBert)', color='orange')
plt.title('Training and Validation Accuracy (DistilBert)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(true_labels, predictions, class_labels):
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.show()

# Example usage after validation
plot_confusion_matrix(true_labels_distilbert, preds_distilbert, class_labels)


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
import matplotlib.pyplot as plt
import numpy as np

# Convert list to numpy arrays
pred_probs_distilbert = np.array(pred_probs_distilbert)
true_labels_distilbert = np.array(true_labels_distilbert)

# Binarize labels for ROC curve
n_classes = len(class_labels)
y_true_bin = label_binarize(true_labels_distilbert, classes=list(range(n_classes)))

# Generate ROC curve
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], pred_probs_distilbert[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves for each class
plt.figure(figsize=(10, 8))
colors = cycle(['blue', 'red', 'green', 'orange', 'purple', 'brown'])

for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Class {class_labels[i]} (area = {roc_auc[i]:0.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class')
plt.legend(loc="lower right")
plt.show()



In [ ]:
from sklearn.metrics import classification_report
# Calculate metrics for DistilBERT
# Ensure the target names match the number of classes in the model output
if len(class_labels) != model_distilbert.num_labels:
	raise ValueError(f"Mismatch between number of classes in model output ({model_distilbert.num_labels}) and target names ({len(class_labels)}). Please update `class_labels` to match the number of classes.")

# Update class_labels to match the number of classes in the model output
class_labels = [str(i) for i in range(model_distilbert.num_labels)]

# Generate classification report
report_distilbert = classification_report(true_labels_distilbert, preds_distilbert, target_names=class_labels)
print("DistilBERT Classification Report:")
print(report_distilbert)

In [ ]:
# Initialize the BERT tokenizer
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# Tokenize data for BERT
def tokenize_data_bert(data):
    return bert_tokenizer(data['Base_Reviews'].tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='pt', return_attention_mask=True)

tokenized_data_bert = tokenize_data_bert(df)


In [ ]:
class TextDatasetBERT(Dataset):
    def __init__(self, inputs, labels, attention_masks):
        self.inputs = inputs
        self.labels = labels
        self.attention_masks = attention_masks

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx].clone().detach().to(torch.long),
            'attention_mask': self.attention_masks[idx].clone().detach().to(torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [ ]:
# Initialize BERT model
model_bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['category'].unique()))
optimizer_bert = AdamW(model_bert.parameters(), lr=2e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_bert.to(device)

# Train model for BERT
def train_model_bert(train_loader, val_loader, epochs):
    # Initialize the learning rate scheduler
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer_bert, num_warmup_steps=0, num_training_steps=total_steps)

    history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': []}
    for epoch in range(epochs):
        model_bert.train()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        for batch in train_loader:
            optimizer_bert.zero_grad()
            inputs = batch['input_ids'].to(device)
            attention_masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model_bert(inputs, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer_bert.step()
            scheduler.step()  # Update learning rate

            total_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += labels.size(0)

        avg_train_loss = total_loss / len(train_loader)
        avg_train_accuracy = correct_predictions / total_samples
        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(avg_train_accuracy)

        # Validation
        model_bert.eval()
        val_loss = 0
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['input_ids'].to(device)
                attention_masks = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model_bert(inputs, attention_mask=attention_masks, labels=labels)
                val_loss += outputs.loss.item()
                preds = torch.argmax(outputs.logits, dim=1)
                correct_predictions += torch.sum(preds == labels).item()
                total_samples += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_accuracy = correct_predictions / total_samples
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(avg_val_accuracy)

        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_accuracy:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}')

    return model_bert, history

In [ ]:
# K-Fold Cross Validation for BERT
kf = KFold(n_splits=5)
accuracies_bert = []
train_losses_bert = []
val_losses_bert = []
train_accuracies_bert = []
val_accuracies_bert = []

for train_idx, val_idx in kf.split(tokenized_data_bert['input_ids']):
    # Split tokenized data
    train_inputs_bert = tokenized_data_bert['input_ids'][train_idx]
    val_inputs_bert = tokenized_data_bert['input_ids'][val_idx]
    train_attention_masks_bert = tokenized_data_bert['attention_mask'][train_idx]
    val_attention_masks_bert = tokenized_data_bert['attention_mask'][val_idx]

    # Labels
    train_labels_bert = df['category'].iloc[train_idx].values
    val_labels_bert = df['category'].iloc[val_idx].values

    # Create datasets
    train_dataset_bert = TextDatasetBERT(train_inputs_bert, train_labels_bert, train_attention_masks_bert)
    val_dataset_bert = TextDatasetBERT(val_inputs_bert, val_labels_bert, val_attention_masks_bert)

    # Create dataloaders
    train_loader_bert = DataLoader(train_dataset_bert, batch_size=16, shuffle=True)
    val_loader_bert = DataLoader(val_dataset_bert, batch_size=16)

    # Train model
    model_bert, history_bert = train_model_bert(train_loader_bert, val_loader_bert, epochs=6)

    # Collect history
    train_losses_bert.extend(history_bert['train_loss'])
    val_losses_bert.extend(history_bert['val_loss'])
    train_accuracies_bert.extend(history_bert['train_accuracy'])
    val_accuracies_bert.extend(history_bert['val_accuracy'])

In [ ]:
# Accuracy calculation
model_bert.eval()
pred_probs_bert, true_labels_bert = [], []
with torch.no_grad():
    for batch in val_loader_bert:
        inputs = batch['input_ids'].to(device)
        attention_masks = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model_bert(inputs, attention_mask=attention_masks)

        # Collect class probabilities
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_probs_bert.extend(probs.cpu().numpy())
        true_labels_bert.extend(labels.cpu().numpy())

# Convert probabilities to predictions
preds_bert = [np.argmax(prob) for prob in pred_probs_bert]

# Calculate accuracy
accuracy_bert = sum([pred == label for pred, label in zip(preds_bert, true_labels_bert)]) / len(true_labels_bert)
accuracies_bert.append(accuracy_bert)

In [ ]:
# Plot training and validation loss/accuracy
plt.figure(figsize=(14, 6))

# Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(train_losses_bert, label='Training Loss (Bert)')
plt.plot(val_losses_bert, label='Validation Loss (Bert)')
plt.title('Training and Validation Loss (Bert)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies_bert, label='Training Accuracy (Bert)')
plt.plot(val_accuracies_bert, label='Validation Accuracy (Bert)', color='orange')
plt.title('Training and Validation Accuracy (Bert)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(true_labels, predictions, class_labels):
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.show()

# Example usage after validation
plot_confusion_matrix(true_labels_bert, preds_bert, class_labels)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
import matplotlib.pyplot as plt
import numpy as np

# Convert list to numpy arrays
pred_probs_bert = np.array(pred_probs_bert)
true_labels_bert = np.array(true_labels_bert)

# Binarize labels for ROC curve
n_classes = len(class_labels)
y_true_bin = label_binarize(true_labels_bert, classes=list(range(n_classes)))

# Generate ROC curve
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], pred_probs_bert[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves for each class
plt.figure(figsize=(10, 8))
colors = cycle(['blue', 'red', 'green', 'orange', 'purple', 'brown'])

for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Class {class_labels[i]} (area = {roc_auc[i]:0.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class')
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import classification_report
# Calculate metrics for BERT
report_bert = classification_report(true_labels_bert, preds_bert, target_names=class_labels)
print("BERT Classification Report:")
print(report_bert)

In [ ]:
# Initialize the RoBERTa tokenizer
roberta_tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
# Tokenize data for RoBERTa
def tokenize_data_roberta(data):
    return roberta_tokenizer(data['Base_Reviews'].tolist(), padding='max_length', truncation=True, max_length=128, return_tensors='pt', return_attention_mask=True)

tokenized_data_roberta = tokenize_data_roberta(df)

In [ ]:
class TextDatasetRoBERTa(Dataset):
    def __init__(self, inputs, labels, attention_masks):
        self.inputs = inputs
        self.labels = labels
        self.attention_masks = attention_masks

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx].clone().detach().to(torch.long),
            'attention_mask': self.attention_masks[idx].clone().detach().to(torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [ ]:
from transformers import get_linear_schedule_with_warmup
# Initialize RoBERTa model
model_roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(df['category'].unique()))
optimizer_roberta = AdamW(model_roberta.parameters(), lr=2e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_roberta.to(device)

# Train model for RoBERTa
def train_model_roberta(train_loader, val_loader, epochs):
    # Initialize the learning rate scheduler
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer_roberta, num_warmup_steps=0, num_training_steps=total_steps)

    history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': []}
    for epoch in range(epochs):
        model_roberta.train()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        for batch in train_loader:
            optimizer_roberta.zero_grad()
            inputs = batch['input_ids'].to(device)
            attention_masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model_roberta(inputs, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer_roberta.step()
            scheduler.step()  # Update learning rate

            total_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += labels.size(0)

        avg_train_loss = total_loss / len(train_loader)
        avg_train_accuracy = correct_predictions / total_samples
        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(avg_train_accuracy)

        # Validation
        model_roberta.eval()
        val_loss = 0
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['input_ids'].to(device)
                attention_masks = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model_roberta(inputs, attention_mask=attention_masks, labels=labels)
                val_loss += outputs.loss.item()
                preds = torch.argmax(outputs.logits, dim=1)
                correct_predictions += torch.sum(preds == labels).item()
                total_samples += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_accuracy = correct_predictions / total_samples
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(avg_val_accuracy)

        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_accuracy:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}')

    return model_roberta, history

In [ ]:
# K-Fold Cross Validation for RoBERTa
kf = KFold(n_splits=5)
accuracies_roberta = []
train_losses_roberta = []
val_losses_roberta = []
train_accuracies_roberta = []
val_accuracies_roberta = []

for train_idx, val_idx in kf.split(tokenized_data_roberta['input_ids']):
    # Split tokenized data
    train_inputs_roberta = tokenized_data_roberta['input_ids'][train_idx]
    val_inputs_roberta = tokenized_data_roberta['input_ids'][val_idx]
    train_attention_masks_roberta = tokenized_data_roberta['attention_mask'][train_idx]
    val_attention_masks_roberta = tokenized_data_roberta['attention_mask'][val_idx]

    # Labels
    train_labels_roberta = df['category'].iloc[train_idx].values
    val_labels_roberta = df['category'].iloc[val_idx].values

    # Create datasets
    train_dataset_roberta = TextDatasetRoBERTa(train_inputs_roberta, train_labels_roberta, train_attention_masks_roberta)
    val_dataset_roberta = TextDatasetRoBERTa(val_inputs_roberta, val_labels_roberta, val_attention_masks_roberta)

    # Create dataloaders
    train_loader_roberta = DataLoader(train_dataset_roberta, batch_size=16, shuffle=True)
    val_loader_roberta = DataLoader(val_dataset_roberta, batch_size=16)

    # Train model
    model_roberta, history_roberta = train_model_roberta(train_loader_roberta, val_loader_roberta, epochs=6)

    # Collect history
    train_losses_roberta.extend(history_roberta['train_loss'])
    val_losses_roberta.extend(history_roberta['val_loss'])
    train_accuracies_roberta.extend(history_roberta['train_accuracy'])
    val_accuracies_roberta.extend(history_roberta['val_accuracy'])

In [ ]:
# Initialize lists to store predicted probabilities and true labels
pred_probs_roberta, true_labels_roberta = [], []

# Switch to evaluation mode
model_roberta.eval()

# Collect probabilities and true labels from the validation set
with torch.no_grad():
    for batch in val_loader_roberta:
        inputs = batch['input_ids'].to(device)
        attention_masks = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model_roberta(inputs, attention_mask=attention_masks)

        # Collect class probabilities
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_probs_roberta.extend(probs.cpu().numpy())

        # Store true labels
        true_labels_roberta.extend(labels.cpu().numpy())

# Convert the probabilities to predictions (class with the highest probability)
preds_roberta = np.argmax(pred_probs_roberta, axis=1)

# Accuracy calculation
accuracy_roberta = sum([pred == label for pred, label in zip(preds_roberta, true_labels_roberta)]) / len(true_labels_roberta)
accuracies_roberta.append(accuracy_roberta)

print(f"Validation Accuracy for Roberta: {accuracy_roberta:.4f}")


In [ ]:
# Plot training and validation loss/accuracy
plt.figure(figsize=(14, 6))

# Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(train_losses_roberta, label='Training Loss (RoBerta)')
plt.plot(val_losses_roberta, label='Validation Loss (RoBerta)')
plt.title('Training and Validation Loss (RoBerta)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies_roberta, label='Training Accuracy (RoBerta)')
plt.plot(val_accuracies_roberta, label='Validation Accuracy (RoBerta)', color='orange')
plt.title('Training and Validation Accuracy (RoBerta)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
import matplotlib.pyplot as plt
import numpy as np

# Convert list to numpy arrays
pred_probs_roberta = np.array(pred_probs_roberta)
true_labels_roberta = np.array(true_labels_roberta)

# Binarize labels for ROC curve
n_classes = len(class_labels)
y_true_bin = label_binarize(true_labels_roberta, classes=list(range(n_classes)))

# Generate ROC curve
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], pred_probs_roberta[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves for each class
plt.figure(figsize=(10, 8))
colors = cycle(['blue', 'red', 'green', 'orange', 'purple', 'brown'])

for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Class {class_labels[i]} (area = {roc_auc[i]:0.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multi-Class')
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(true_labels, predictions, class_labels):
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')
    plt.show()

# Example usage after validation
plot_confusion_matrix(true_labels_roberta, preds_roberta, class_labels)

In [ ]:
from sklearn.metrics import classification_report
# Calculate metrics for ROBERTA
report_roberta = classification_report(true_labels_roberta, preds_roberta, target_names=class_labels)
print("ROBERTA Classification Report:")
print(report_roberta)

In [ ]:
# Print accuracies
print(f"BERT Accuracy: {accuracy_bert:.4f}")
print(f"DistilBERT Accuracy: {accuracy_distilbert:.4f}")
print(f"RoBERTa Accuracy: {accuracy_roberta:.4f}")

In [ ]:
# Plot accuracies
import seaborn as sns
import matplotlib.pyplot as plt

# Set the style for the plot
sns.set(style="whitegrid")


# Data for plotting
model_names = ['BERT', 'DistilBERT', 'RoBERTa']
accuracies = [accuracy_bert, accuracy_distilbert, accuracy_roberta]

# Create a DataFrame
data = pd.DataFrame({'Model': model_names, 'Accuracy': accuracies})

# Plot using seaborn
plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Accuracy', data=data, palette='Set2')

# Customize the plot
plt.title('Model Accuracy Comparison')
plt.ylim(0, 1)  # Accuracy range
plt.show()